In [1]:
# # 下載資料
# !gdown --id '14CqX3OfY9aUbhGp4OpdSHLvq2321fUB7' --output data.zip
# # 解壓縮
# !unzip data.zip

In [2]:
import os
# 讀取 label.csv
import pandas as pd
# 讀取圖片
from PIL import Image
import numpy as np

import torch
# Loss function
import torch.nn.functional as F
# 讀取資料
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
# 載入預訓練的模型
import torchvision.models as models
# 將資料轉換成符合預訓練模型的形式
import torchvision.transforms as transforms
# 顯示圖片
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using", "cuda" if torch.cuda.is_available() else "cpu")

using cuda


In [3]:
# 實作一個繼承 torch.utils.data.Dataset 的 Class 來讀取圖片
class Adverdataset(Dataset):
    def __init__(self, root, label, transforms):
        # 圖片所在的資料夾
        self.root = root
        # 由 main function 傳入的 label
        self.label = torch.from_numpy(label).long()
        # 由 Attacker 傳入的 transforms 將輸入的圖片轉換成符合預訓練模型的形式
        self.transforms = transforms
        # 圖片檔案名稱的 list
        self.fnames = []

        for i in range(200):
            self.fnames.append("{:03d}".format(i))

    def __getitem__(self, idx):
        # 利用路徑讀取圖片
        img = Image.open(os.path.join(self.root, self.fnames[idx] + '.png'))
        # 將輸入的圖片轉換成符合預訓練模型的形式
        img = self.transforms(img)
        # 圖片相對應的 label
        label = self.label[idx]
        return img, label
    
    def __len__(self):
        # 由於已知這次的資料總共有 200 張圖片 所以回傳 200
        return 200

In [4]:
class Classfier:
    def __init__(self, img_dir, label):
        # 讀入預訓練模型 vgg16
        self.model = models.densenet121(pretrained=True)
        self.model.to(DEVICE)
        self.model.eval()
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]
        # 把圖片 normalize 到 0~1 之間 mean 0 variance 1
        self.normalize = transforms.Normalize(self.mean, self.std, inplace=False)
        transform = transforms.Compose([
            transforms.Resize((224, 224), interpolation=3),
            transforms.ToTensor(),
            self.normalize
        ])
        # 利用 Adverdataset 這個 class 讀取資料
        self.dataset = Adverdataset(img_dir, label, transform)
        
        self.loader = torch.utils.data.DataLoader(
            self.dataset,
            batch_size = 1,
            shuffle = False,
#             num_workers = 8
        )
    
    def predict(self):
        # 存下一些成功攻擊後的圖片 以便之後顯示
        results = []
        for (data, target) in self.loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = self.model(data)
            pred = output.max()
            results.append(pred)
        return results



In [5]:
# 讀入圖片相對應的 label
df = pd.read_csv("../data/labels.csv")
df = df.loc[:, 'TrueLabel'].to_numpy()
label_name = pd.read_csv("../data/categories.csv")
label_name = label_name.loc[:, 'CategoryName'].to_numpy()
# new 一個 Attacker class
attacker = Classfier('../data/images', df)

attacker.predict()

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.53 GiB already allocated; 11.14 MiB free; 39.13 MiB cached)

In [ ]:
# %matplotlib inline
# # 顯示圖片
# cnt = 0
# plt.figure(figsize=(30, 30))
# for i in range(len(epsilons)):
#     try:
#         os.mkdir(f'results/{epsilons[i]}')
#     except FileExistsError:
#         pass
#     for j, img in enumerate(results[i]):
#         img = img[3] * 255
#         img = np.clip(img, 0, 255)
#         img = img.astype('uint8')
#         img = np.transpose(img, (1, 2, 0))
#         img = Image.fromarray(img, 'RGB')
#         img.save(f'results/{epsilons[i]}/%03d.png' % j)

#     for j in range(len(examples[i])):
#         cnt += 1
#         plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
#         plt.xticks([], [])
#         plt.yticks([], [])
#         if j == 0:
#             plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
#         orig, adv, orig_img, ex = examples[i][j]
#         # plt.title("{} -> {}".format(orig, adv))
#         plt.title("original: {}".format(label_name[orig].split(',')[0]))
#         orig_img = np.transpose(orig_img, (1, 2, 0))
#         plt.imshow(orig_img)
#         cnt += 1
#         plt.subplot(len(epsilons),len(examples[0]) * 2,cnt)
#         plt.title("adversarial: {}".format(label_name[adv].split(',')[0]))
#         ex = np.transpose(ex, (1, 2, 0))
#         ex = ex * 255
#         ex = np.clip(ex, 0, 255)
#         ex = ex.astype('uint8')
#         plt.imshow(ex)
# plt.tight_layout()
# plt.show()

In [ ]:
torch.cuda.empty_cache()